In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI, BadRequestError
import gradio as gr
from pypdf import PdfReader
import telebot
import json

In [ ]:
# load environment variables from .env file
load_dotenv(override=True)

# load OpenAI API key and base URL from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_API_BASE")
openai_default_model = os.getenv("OPENAI_DEFAULT_MODEL")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in environment variables.")

# initialize OpenAI client
openai = OpenAI(api_key=openai_api_key)

if openai_base_url:
    openai.base_url = openai_base_url

In [ ]:
# configure telegram bot
telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
if not telegram_bot_token:
    raise ValueError("TELEGRAM_BOT_TOKEN is not set in environment variables.")

bot = telebot.TeleBot(telegram_bot_token)


In [ ]:
chat_id = os.getenv("TELEGRAM_CHAT_ID")
if not chat_id:
    raise ValueError("TELEGRAM_CHAT_ID is not set in environment variables.")

def send_telegram_message(text):
    print(f"Sending telegram message to chat_id {chat_id}: {text}")
    bot.send_message(chat_id, text)

In [ ]:
# send_telegram_message("Bot has started successfully.")

In [ ]:
def record_user_details(email, name="Not provided", notes="Not provided"):
    send_telegram_message(f"New user details recorded: \nName: {name} \nEmail: {email} \nNotes: {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    send_telegram_message(f"I coudn't answer the following question: \n{question}")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
# test a tool call
# globals()["record_unknown_question"]("this is a really hard question")

In [ ]:
# extract information from resume PDF
reader = PdfReader("../resources/resume.pdf")
reseume_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        reseume_text += text

In [ ]:
reseume_text

In [ ]:
summary_text = ""
with open("../resources/summary.txt", "r", encoding="utf-8") as f:
    summary_text = f.read()

In [ ]:
name = "Asir Mosaddek Sakib"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary_text}\n\n## Resume:\n{reseume_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
system_prompt

In [ ]:
def chat(message, chat_history):
    # if(not chat_history or len(chat_history) == 0):
    #     # a new conversation, send telegram message
    #     send_telegram_message("New conversation started on the chatbot.")
    
    # build messages for the OpenAI chat API using the stored chat_history (list of (user, bot) tuples)
    history = [{"role": h["role"], "content": h["content"]} for h in chat_history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    content = ""
    done = False
    while not done:
        try:
            response = openai.chat.completions.create(
                model=openai_default_model if openai_default_model else "gpt-4.1-mini",
                messages=messages,
                tools=tools
            )

            finish_reason = response.choices[0].finish_reason

            if finish_reason=="tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = handle_tool_calls(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
                content = response.choices[0].message.content
            
        except BadRequestError as e:
            content = "Something went wrong. Please try again later."
            print(f"BadRequestError: {e}")
        except Exception as e:
            content = "Something went wrong. Please try again later."
            print(f"Error: {e}")
    return content


In [ ]:
demo = gr.ChatInterface(fn=chat, title="Asir's Chatbot")
demo.launch(share = False)